In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from pathlib import Path

import geopandas as gpd
import numpy as np
import plotly.express as px
import pandas as pd

from paris_bikes.preprocess_data import get_parkings_per_iris, get_population_per_iris

## Load files

In [ ]:
census_filepath = Path("../data/raw/RECENSEMENT_IRIS_POPULATION.geojson")
parking_filepath = Path("../data/raw/stationnement-voie-publique-emplacements.geojson")
green_spaces_filepath = Path("../data/raw/espaces_verts.geojson")

df_census = gpd.read_file(census_filepath)
df_parking = gpd.read_file(parking_filepath)
df_green = gpd.read_file(green_spaces_filepath).set_index("nom_ev")

## Get parking spots per IRIS

IRIS is a unit of area, defined [here](https://www.insee.fr/en/metadonnees/definition/c1523).

Using only bike parking spots, where Régime Particulier `regpar` is in `["Vélos", "Box à vélos"]`.

In [ ]:
df_iris = get_population_per_iris(df_census)
df_parks_per_iris = get_parkings_per_iris(df_parking, df_iris)

# Compute parking spots per person (per IRIS)
df_merge = df_iris.join(df_parks_per_iris)
df_merge["parks_per_person"] = (
    df_merge["nb_parking_spots"] / df_merge["nb_pop"]
)

## Overview

In [ ]:
df_merge["nb_parking_spots"].isnull().sum()

In [ ]:
df_merge["parks_per_person"].describe()

## Tops and bottoms

In [ ]:
print(
    df_merge
    .sort_values("parks_per_person")
    .loc[:, ["nb_pop", "nb_parking_spots", "parks_per_person"]]
    .head(10)
)

In [ ]:
print(
    df_merge
    .sort_values("parks_per_person")
    .loc[:, ["nb_pop", "nb_parking_spots", "parks_per_person"]]
    .tail(10)
)

In [ ]:
print(
    df_merge
    .loc[df_merge["nb_pop"] > 0]
    .sort_values("parks_per_person")
    .loc[:, ["nb_pop", "nb_parking_spots", "parks_per_person"]]
    .tail(10)
)

## Choropleth

In [ ]:
df_plot = df_merge[["parks_per_person", "geometry"]].copy()
df_plot["parks_per_person"].clip(
    upper=df_plot["parks_per_person"].quantile(0.95),
    inplace=True
)
fig = px.choropleth(
    df_plot, 
    geojson=df_plot.geometry, 
    locations=df_plot.index, 
    projection="mercator", 
    color="parks_per_person"
)
fig.update_geos(fitbounds="locations", visible=True)
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

## Histogram

In [ ]:
df_plot.hist()

## Green spaces

In [ ]:
df_green.head()

In [ ]:
# Filter only relevant green spaces
scope_type_ev = [
    'Promenades ouvertes',
    'Cimetières',
    'Bois',
]
df_green_filtered = df_green.loc[
    df_green.type_ev.isin(scope_type_ev) &
    df_green.adresse_codepostal.str.startswith("75")
].copy()

# Define a 100 m buffer around each green space
green_space_buffer = gpd.GeoDataFrame(geometry=(
    df_green_filtered
    .to_crs(epsg=2154)
    .geometry
    .buffer(100)
    .to_crs(epsg=4326)
))

# Identify bike parkings within each green space + buffer
df_parkings_in_green_spaces = green_space_buffer.sjoin(df_parking, how="inner")

# Get the total parking spots per green area
df_green_filtered["parkings"] = df_parkings_in_green_spaces.groupby("nom_ev")["plarel"].sum()

# Compute parking spots per person sqrt of area
df_green_filtered["parkings_per_sqrt_area"] = (
    df_green_filtered["parkings"] / np.sqrt(df_green_filtered["poly_area"])
)
df_green_filtered["parkings_per_perimeter"] = (
    df_green_filtered["parkings"] / df_green_filtered["perimeter"]
)

In [ ]:
df_green_filtered["parkings_per_perimeter"].dropna().sort_values()

In [ ]:
df_green_filtered["parkings_per_perimeter"].replace([np.inf, -np.inf], np.nan).dropna().hist()

In [ ]:
df_plot = df_green_filtered[["parkings_per_perimeter", "geometry"]].copy()
df_plot["parkings_per_perimeter"].clip(
    upper=df_plot["parkings_per_perimeter"].quantile(0.8),
    inplace=True
)
fig = px.choropleth(
    df_plot, 
    geojson=df_plot.geometry, 
    locations=df_plot.index, 
    projection="mercator", 
    color="parkings_per_perimeter"
)
fig.update_geos(fitbounds="locations", visible=True)
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

### Green spaces by IRIS

In [ ]:
# Join with IRIS and aggregate results at IRIS level
iris_join = (
    df_green_filtered
    .sjoin(df_census.reset_index().loc[:, ["l_ir", "geometry"]], how="left")
    .groupby("l_ir")
    .agg({'perimeter': 'sum', "poly_area": "sum", "parkings": "sum"})
)

# # Compute parking spots per person sqrt of area
iris_join["parkings_per_sqrt_area"] = (
    iris_join["parkings"] / np.sqrt(iris_join["poly_area"])
)
iris_join["parkings_per_perimeter"] = (
    iris_join["parkings"] / iris_join["perimeter"]
)

# # Get intersection with IRIS
# buffer_intersection = (
#     green_space_buffer.reset_index()
#     .overlay(df_census.reset_index().loc[:, ["l_ir", "geometry"]], how='intersection')
# )
# green_space_intersection = (
#     df_green_filtered.reset_index()
#     .overlay(df_census.reset_index().loc[:, ["l_ir", "geometry"]], how='intersection')
# )

# # Identify the IRIS of each parking spot
# df_bike_parking = df_parking.loc[df_parking.regpar.isin(["Vélos", "Box à vélos"])]
# df_parks_with_iris = df_bike_parking.sjoin(df_census.loc[:, ["nb_pop", "geometry"]], how="inner")

# # Get the total parking spots per IRIS
# parks_per_iris = df_parks_with_iris.groupby("index_right")["plarel"].sum()

# # Compute parking spots per person (per IRIS)
# df_census_with_parks = df_census.join(parks_per_iris)
# df_census_with_parks["parks_per_person"] = (
#     df_census_with_parks["plarel"] / df_census_with_parks["nb_pop"]
# )

# # Identify bike parkings within each green space + buffer
# df_parkings_in_green_spaces = green_space_buffer.sjoin(df_parking, how="inner")

# # Get the total parking spots per green area
# df_green_filtered["parkings"] = df_parkings_in_green_spaces.groupby("nom_ev")["plarel"].sum()



In [ ]:
iris_join.shape

In [ ]:
iris_join["geometry"] = (
    df_census
    .geometry
    .loc[iris_join.index]
    .reset_index()
    .drop_duplicates(subset="l_ir", keep='first')
    .set_index("l_ir")
)
iris_join = gpd.GeoDataFrame(iris_join)

In [ ]:
iris_join

In [ ]:
fig = px.choropleth(
    iris_join, 
    geojson=iris_join.geometry, 
    locations=iris_join.index, 
    projection="mercator", 
    color="parkings_per_perimeter"
)
fig.update_geos(fitbounds="locations", visible=True)
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()